In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/garanti-bbva-data-camp/work_experiences.csv
/kaggle/input/garanti-bbva-data-camp/languages.csv
/kaggle/input/garanti-bbva-data-camp/submission.csv
/kaggle/input/garanti-bbva-data-camp/test_users.csv
/kaggle/input/garanti-bbva-data-camp/skills.csv
/kaggle/input/garanti-bbva-data-camp/train_users.csv
/kaggle/input/garanti-bbva-data-camp/education.csv


Kütüphanelerin yüklenmesi ve bazı ayarlar 


In [2]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import unicodedata
import warnings
import re 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, validation_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier

In [3]:
warnings.simplefilter(action='ignore', category=Warning)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 30);
pd.set_option('display.width', 300)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [4]:
#Verilerin okunması 
work_experiences = pd.read_csv('/kaggle/input/garanti-bbva-data-camp/work_experiences.csv')
languages = pd.read_csv('/kaggle/input/garanti-bbva-data-camp/languages.csv')
submission = pd.read_csv('/kaggle/input/garanti-bbva-data-camp/submission.csv')
test_users = pd.read_csv('/kaggle/input/garanti-bbva-data-camp/test_users.csv')
skills = pd.read_csv('/kaggle/input/garanti-bbva-data-camp/skills.csv')
train_users = pd.read_csv('/kaggle/input/garanti-bbva-data-camp/train_users.csv')
education = pd.read_csv('/kaggle/input/garanti-bbva-data-camp/education.csv')

In [5]:
#ana dataframe lerin indexleri user_id olarak ayarlandı
train_users = train_users.set_index("user_id")
test_users = test_users.set_index("user_id")
submission = submission.set_index("user_id")

In [6]:
#string ifadelerin daha kolay analiz edilmesi için bir fonksiyon.
def turkish_to_english(text):
    # Türkçe karakterleri İngilizce karşılıkları ile değiştir
    text = re.sub("[çÇ]", "c", text)
    text = re.sub("[ğĞ]", "g", text)
    text = re.sub("[ıİ]", "i", text)
    text = re.sub("[öÖ]", "o", text)
    text = re.sub("[şŞ]", "s", text)
    text = re.sub("[üÜ]", "u", text)
    
    # Diğer Unicode karakterlerini kaldır
    return ''.join(char for char in unicodedata.normalize('NFKD', text)
                   if not unicodedata.combining(char))


# **users bilgilerine genel bakış ve birleştirme**

* **kullanıcılarda yapılacak işlemleri kolaylaştırmak için birleştirme yapıyoruz**

In [7]:

df = pd.concat([train_users,test_users],axis=0)
df.head()

,industry,location,moved_after_2019
user_id,,,
1301,Information Technology and Services,"Istanbul, Istanbul, Turkey",1.00000
6950,Internet,"Istanbul, Istanbul, Turkey",0.00000
4880,Online Media,Turkey,0.00000
26046,Telecommunications,"Istanbul, Istanbul, Turkey",0.00000
11005,Banking,"Istanbul, Turkey",0.00000


In [8]:
#lokasyon bilgisinin aynı anlama gelse de farklı ifadelerle belirtilmesini düzeltmek için bazı işlemler 
df["location"]= df["location"].astype('str')
df["location"]= df["location"].str.lower()
df["location"]= df["location"].apply(turkish_to_english)

In [9]:
#'location' değişkeninin kaç tane eşsiz değere sahip olduğuna bakalım (434)
df["location"].nunique()


430

In [10]:
#buradaki fazlalık aynı şehri farklı şekillerde ifade etmekten kaynaklanıyor gibi 
#çözüm için türkiyedeki tüm illeri sadece ismiyle belirtecek şekilde standartlaştırma yapalım 

In [11]:
turkish_cities = ["adana", "adiyaman", "afyon", "agri", "amasya", "ankara", "antalya", "artvin", "aydin", 
                  "balikesir", "bilecik", "bingol", "bitlis", "bolu", "burdur", "bursa", "canakkale", "cankiri", 
                  "corum", "denizli", "diyarbakir", "edirne", "elazig", "erzincan", "erzurum", "eskisehir", "antep", 
                  "giresun", "gumushane", "hakkari", "hatay", "isparta", "mersin", "istanbul", "izmir", "kars", "kastamonu", 
                  "kayseri", "kirklareli", "kirsehir", "kocaeli", "konya", "kutahya", "malatya", "manisa", "maras", 
                  "mardin", "mugla", "mus", "nevsehir", "nigde", "ordu", "rize", "sakarya", "samsun", "siirt", "sinop", "sivas", 
                  "tekirdag", "tokat", "trabzon", "tunceli", "urfa", "usak", "van", "yozgat", "zonguldak", "aksaray", "bayburt", 
                  "karaman", "kirikkale", "batman", "sirnak", "bartin", "ardahan", "igdir", "yalova", "karabuk", "kilis", "osmaniye", "duzce"]

In [12]:
#şehir ismin göre standartlaştırma işlemi
for city in turkish_cities:
    df["location"] = df["location"].apply(lambda x: city if city in x else x)
    

In [13]:
#bu sayede eşsiz değer sayımız oldukça düşecektir. (122)
df["location"].nunique()

122

In [14]:
# ;'industry' değişkeni için de küçültle ve unicode işlemlerini yapalım 
df["industry"]= df["industry"].astype('str')
df["industry"]= df["industry"].str.lower()
df["industry"]= df["industry"].apply(turkish_to_english)

In [15]:
#burada locaion benzeri bir standrtlaştırma zor olacağından ve eşsiz değer sayısı fazla olmadığından bu şekilde bırakmayı tercih ediyorum
df["industry"].nunique()

142

* work_exp datasına genel bakış ve düzenlemeler 

In [16]:
#öncelikle kaç iş değiştirdiğini df e ekleyelim
df["exp_count"] = work_experiences.groupby('user_id')['company_id'].count()
df["exp_count"].fillna(0,inplace=True)

In [17]:
#işe başlama tarihini datetime olarak work_experiences datasına ekleyelim ekleyelim
work_experiences['start_date'] = pd.to_datetime(work_experiences['start_year_month'].astype(str), format='%Y%m')
work_experiences.drop(["start_year_month"],axis=1)

,user_id,company_id,location,start_date
0,53442,2651,"Istanbul, Turkey",2015-09-01
1,34558,815,"Istanbul, Turkey",2012-10-01
2,63761,26354,NaN,2000-10-01
3,10738,89,NaN,2016-10-01
4,8711,3113,"Istanbul, Turkey",2018-01-01
...,...,...,...,...
187014,22180,15065,Gebze,2012-05-01
187015,55822,25076,stajer,2015-06-01
187016,13750,1607,NaN,2019-01-01
187017,3679,1414,"Ankara, Turkey",2018-07-01


In [18]:
#her bir kişinin en son başladığı işin tarihini tutalım
latest_start_dates = work_experiences.groupby('user_id')['start_date'].max()

tarih = pd.datetime(2019,1,1)

#burada kişinin en son işinde ne kadardır kaldığı yazıyor.
last_job_d = tarih - latest_start_dates
df["last_job_m"] = last_job_d / np.timedelta64(1, 'M')

In [19]:
#bu fonksiyon ile kişinin ortalama işte kalma süresini hesaplayalım
def average_job_tenure(df):
  df = df.sort_values(by='start_date')
  df['tenure'] = df.groupby('user_id')['start_date'].diff()
  df['tenure'] = df['tenure'] / np.timedelta64(1, 'M')
  avg_tenure = df.groupby('user_id')['tenure'].mean()
  return avg_tenure

df["average_work"] = average_job_tenure(work_experiences)


* Languages datasına genel bakış ve düzenlemeler 

In [20]:
languages['language'] = languages['language'].str.lower()
languages['language'] = languages['language'].apply(turkish_to_english)

In [21]:
languages['language'] = languages['language'].apply(lambda x: 'ing'
                                                    if 'english' in x
                                                    or 'ing' in x
                                                    or 'englisch'in x
                                                    or 'engilish'in x
                                                    or 'ingilizce'in x
                                                    or 'ielts'in x
                                                    else x)
languages['language'].value_counts()['ing']

37165

In [22]:
languages['language'] = languages['language'].apply(lambda x: 'tr'
                                                    if 'turkish' in x
                                                    or 'turkısh' in x
                                                    or 'türkçe' in x
                                                    or 'turk' in x
                                                    or 'turkce' in x
                                                    or 'turkisch' in x
                                                    or 'turksih' in x
                                                    else x)
languages['language'].value_counts()['tr']

22623

In [23]:
languages['language'] = languages['language'].apply(lambda x: 'ger'
                                                    if 'german' in x
                                                    or 'alman' in x
                                                    or 'deutsch'in x
                                                    else x)
languages['language'].value_counts()['ger']

7188

In [24]:
languages['language'] = languages['language'].apply(lambda x: 'jp'
                                                    if 'japanese' in x
                                                    or 'japonca' in x
                                                    else x)
languages['language'].value_counts()['jp']

442

In [25]:
languages['language'] = languages['language'].apply(lambda x: 'fr'
                                                    if 'fransızca' in x
                                                    or 'french' in x
                                                    or 'francais' in x
                                                    or 'fransizca' in x
                                                    or 'france' in x
                                                    else x)
languages['language'].value_counts()['fr']

1752

In [26]:
#bildiği dilin seviyesini sayısal hale dönüştürme.
proficiency_encod = {
    'elementary': 1,
    'limited_working': 2,
    'professional_working': 3,
    'full_professional': 4,
    'native_or_bilingual': 5
}

languages['proficiency'] = languages['proficiency'].map(proficiency_encod)

In [27]:
languages["language"].value_counts()

ing                       37165
tr                        22623
ger                        7188
fr                         1752
ispanyolca                  860
                          ...  
oracle data integrator        1
slovakca                      1
pushto                        1
indonesian                    1
grek                          1
Name: language, Length: 273, dtype: int64

In [28]:

languages_summary = languages.loc[languages['language'] == 'ing'].groupby('user_id').agg({'proficiency': 'max'})


In [29]:
df = df.merge(languages_summary, on='user_id', how='left')

In [30]:
#diğer dillerin skorunun toplamayı tercih ettik
proficiency = languages[languages['language'] != 'ing'].groupby('user_id').agg({'proficiency': 'sum'})
df = df.merge(proficiency, on='user_id', how='left')


* Skılls datasına genel bakış ve düzenlemeler 

In [31]:
skills['skill'] = skills['skill'].str.lower()
skills['skill'] = skills['skill'].apply(turkish_to_english)

In [32]:
#belirlediğimiz anahtar kelimelere göre bazı yetenekleri gruplandıralım
soft_skills = ['problem', 'yarat', 'yenilik', 'dusunme', 'lider', ' yonetim',
               'ekip', 'iletisim', 'dogru', 'organizasyon', 'plan', 'takim','cozme',
               'egitim	','arastırma', 'analiz','analysis','management','collaboration','yetenek',
               'research','dusun','strateji','ekip','presentation','yonetim','team','leadership','analitik','analytical',
               'speaking','communication']

data_skills = ['data','veri','makine','machine','ai','keras','sql','nlp','hadoop','artificial intelligence','oracle','apache','kafka',
              'mongodb','redis','deep','opencv','goruntu','image','subversion']

office = ['office','microsoft','powerpoint','excel']

languages = ['english','almanca','turkce','ispanyolca','ingilizce','german']

web = ['web','network','rest','html','jquery','tcp','bootstrap','angular',
      'html', 'css', 'javascript', 'php','ajax', 'react', 'vue.js', 'node.js', 'express.js', 'laravel', 'django',
       'flask', 'ruby','json', 'soap','graphql','oauth', 'jwt','asp.net','wordpress','xml','selenium','seo','jsp',
      'tomcat','firebase','nodejs']

cloud= ['aws','bulut','cloud','azure','openstack','saas', 'paas']

p_languages = ['javascript','php','c++','c#','python','css','kotlin','java','typescript','ruby', 'swift','perl','scala'
               ,'sql', 'html','objective-c']

programming = ['programming','software','develop','algorit','algorithms','gelistirme','oop','oriented','git','linux'
               ,'programlama','maven','docker','linq']

mobil = ['kotlin','iphone','ios','mobile','app','swift','android','flutter','mobil','firebase']

engineering =['matlab','engineer','solid',"autocad","ansys","labview", "catia", "creo", "comsol", "revit",'muhendis']

tree_d = ['blender','3d','3b','unity']

adobe = ['adobe','photoshop','premier']

ide = ['vscode','pycharm','sublime','eclipse','netbeans','intellij','xcode','jupyter','visual']

framework = ['framework','django','flask','pyramid','tornado','cherryPy',
            'react','angularjs','vue.js','ember.js','backbone.js',
            'spring','hibernate','gwt','struts',
             'sinatra','hanami','grape','padrino','laravel','.net','silverlight']

dev_methods = ['agile','scrum','kanban','lean','extreme programming','jira','uml','design patterns','tfs','jenkins','mvc']

business = ['crm','business','musteri','marketing','ar-ge','commerce','ticaret','customer']

cyber_sec = ['security','siber','cyber']

virtual_tec = ['docker','vmware','sanal','kubernetes','hyper-v','virtualbox']

os_ = ['linux','unix','bash','ubuntu','windows']

In [33]:
#tüm gruplandırmayı tek seferde yapabilmek için bu grupları sözlük yapısı ile aldım
clusters = {'adobe':adobe,'tree_d':tree_d,'engineering':engineering,
            'mobil':mobil,'p_languages':p_languages,
            'cloud':cloud
            ,'web':web
            ,'office':office
            ,'data_skills':data_skills
            ,'soft_skills':soft_skills
           ,'programming':programming
           ,'languages':languages
           ,'ide':ide
           ,'framework':framework
           ,'dev_methods':dev_methods,
           'business':business
           ,'cyber_sec':cyber_sec
           ,'virtual_tec':virtual_tec
           ,'os_':os_}

In [34]:
#gruplandırmayı yapacak fonksiyon
def skills_groping (df,cat_name,cat_list):
    df[cat_name] = skills['skill'].apply(lambda x: 1 if any(skill in x for skill in cat_list) else 0)
    return df

for name,cluster in clusters.items():
    skills = skills_groping(skills,str(name)+'_check',cluster)

In [35]:
#içinde 'C' ve 'R' geçen bütün becerileri p_languages olarak değerlendirmek istemediğimiz için 'C' ve 'R' diline özel bunu yazmayı tercih ettik.
skills.loc[skills['skill'] == 'c', 'p_languages_check'] = 1
skills.loc[skills['skill'] == 'r', 'data_skills_check'] = 1

In [36]:
#kişinin ilgili beceri grubundaki toplam beceri sayısını df e ekledik
columns = list(skills.columns)
for col in columns[2:]:
    df[str(col)+'_sum'] = skills.groupby('user_id').agg({col: 'sum'})
    df[str(col)+'_sum'].fillna(0,inplace=True)

In [37]:
#belirtilen toplam yetenek sayısını ekleyelim belirtmeyenlere 0 girelim.
df["skills_count"] = skills.groupby('user_id')['skill'].nunique()
df["skills_count"].fillna(0,inplace=True)

* Education datasına genel bakış ve düzenlemeler 

In [38]:
education = education.applymap(lambda x: x.lower() if type(x) == str else x)
education = education.applymap(lambda x: turkish_to_english(x) if isinstance(x, str) else x)
education['degree'] = education['degree'].astype("str")

In [39]:
#fazla eksik değer var silindi
education.drop(["start_year_month",'end_year_month'],axis=1,inplace=True)


In [40]:
#universite ya da lise olmasını kontrol etme 
education['uni_check'] = education['school_name'].apply(lambda x: 1 if "universite" in x or 'university' in x or "enstitu" in x or "institute" in x
                                                    else 0)
education['h_s_check'] = education['school_name'].apply(lambda x: 1 if "lise" in x or 'high' in x or "kolej" in x or "college" in x
                                                    else 0)

In [41]:
#toplam gittiği üniversite sayısını aldık.
df["uni_sum"] = education.groupby('user_id').agg({"uni_check": 'sum'})
df["uni_sum"].fillna(0,inplace=True)

field_of_study

In [42]:
education["fields_of_study"]= education["fields_of_study"].astype('str')
education["fields_of_study"]= education["fields_of_study"].str.lower()
education["fields_of_study"]= education["fields_of_study"].apply(turkish_to_english)

In [43]:
education["fields_of_study"].value_counts().iloc[:50]

nan                                           19838
bilgisayar muhendisligi                       14748
computer engineering                          12746
elektrik ve elektronik muhendisligi            3564
electrical and electronics engineering         3224
                                              ...  
computer engineer                               352
electronics and communications engineering      351
engineering management                          339
civil engineering                               336
fizik                                           323
Name: fields_of_study, Length: 50, dtype: int64

-*-*-*-*-*-*-*-*-*-*Encoding -*-*-*-*-*-*-*-*-

In [44]:
columns_last = list(df.columns)

In [45]:
#sayısal değişkenlerin NaN değerlerini 0 ile dolduralım 
for col in columns_last[3:]:
    df[col].fillna(0,inplace=True)

In [46]:
#kategorik değişkenlerin çok fazla eşsiz değerleri olduğundan, model karmaşıklığını arttırmamak için Rare encoding uygulayacaız 
df['location'].nunique() 
df['industry'].nunique()

142

In [47]:
def rare_encoder(dataframe, rare_perc, cat_cols):
    rare_columns = [col for col in cat_cols if (dataframe[col].value_counts() / len(dataframe) < 0.01).sum() > 1]

    for col in rare_columns:
        tmp = dataframe[col].value_counts() / len(dataframe)
        rare_labels = tmp[tmp < rare_perc].index
        dataframe[col] = np.where(dataframe[col].isin(rare_labels), 'Rare', dataframe[col])

    return dataframe

In [48]:

df = rare_encoder(df,0.0015,['location'])
df['location'].nunique()  #16
df = rare_encoder(df,0.0015,['industry'])
df['industry'].nunique() #51

51

In [49]:
#encoding yapmak için kategorik değişkenleri alalım 
cat_cols =["location","industry"]
df_encoded = pd.get_dummies(df[cat_cols], columns=cat_cols,drop_first=True)

In [50]:
#veriye ekleyelim 
df = pd.concat([df, df_encoded], axis=1, join='outer')

Model kurulması

seçeceğimiz modele karar vermek için, bize verilen eğitim verisini kendi içinde eğitim ve test diye ayırıp bir kaç model kuralım

In [51]:
train = df.loc[train_users.index, df.columns]
test = df.loc[test_users.index, df.columns]
drop_list = ["industry","location","moved_after_2019"]
X_t = train.drop(drop_list,axis = 1)
y_t = train['moved_after_2019']

In [52]:
"""def all_model():
    #veriyi test ve train olarak bölme 
    X_train, X_test, Y_train, Y_test = train_test_split(X_t, y_t, test_size=0.25, random_state=46)
    # LOjistik Regresyon

    logreg = LogisticRegression()
    logreg.fit(X_train, Y_train)
    acc_log = round(logreg.score(X_test, Y_test) * 100, 2)
    acc_log

    # Support Vector Machines
    svc = SVC()
    svc.fit(X_train, Y_train)
    Y_pred = svc.predict(X_test)
    acc_svc = round(svc.score(X_test, Y_test) * 100, 2)
    acc_svc

    #KNN 
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    acc_knn = round(knn.score(X_test, Y_test) * 100, 2)
    acc_knn

    # Decision Tree
    decision_tree = DecisionTreeClassifier()
    decision_tree.fit(X_train, Y_train)
    Y_pred = decision_tree.predict(X_test)
    acc_decision_tree = round(decision_tree.score(X_test, Y_test) * 100, 2)
    acc_decision_tree

    # Random Forest
    random_forest = RandomForestClassifier(n_estimators=200, n_jobs=-1)
    random_forest.fit(X_train, Y_train)
    Y_pred = random_forest.predict(X_test)
    random_forest.score(X_train, Y_train)
    acc_random_forest = round(random_forest.score(X_test, Y_test) * 100, 2)
    acc_random_forest

    models = pd.DataFrame({
        'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression',
                  'Random Forest', 'Decision Tree'],
        'Score': [acc_svc, acc_knn, acc_log,
                  acc_random_forest,acc_decision_tree]})
    return models
    """

"def all_model():\n    #veriyi test ve train olarak bölme \n    X_train, X_test, Y_train, Y_test = train_test_split(X_t, y_t, test_size=0.25, random_state=46)\n    # LOjistik Regresyon\n\n    logreg = LogisticRegression()\n    logreg.fit(X_train, Y_train)\n    acc_log = round(logreg.score(X_test, Y_test) * 100, 2)\n    acc_log\n\n    # Support Vector Machines\n    svc = SVC()\n    svc.fit(X_train, Y_train)\n    Y_pred = svc.predict(X_test)\n    acc_svc = round(svc.score(X_test, Y_test) * 100, 2)\n    acc_svc\n\n    #KNN \n    knn = KNeighborsClassifier(n_neighbors=3)\n    knn.fit(X_train, Y_train)\n    Y_pred = knn.predict(X_test)\n    acc_knn = round(knn.score(X_test, Y_test) * 100, 2)\n    acc_knn\n\n    # Decision Tree\n    decision_tree = DecisionTreeClassifier()\n    decision_tree.fit(X_train, Y_train)\n    Y_pred = decision_tree.predict(X_test)\n    acc_decision_tree = round(decision_tree.score(X_test, Y_test) * 100, 2)\n    acc_decision_tree\n\n    # Random Forest\n    random_fo

In [53]:
X, y = train.drop(columns=["moved_after_2019","industry","location"]), train["moved_after_2019"]

In [54]:
test.drop(["industry","location"],axis=1,inplace=True)
test.drop(["moved_after_2019"],axis=1,inplace=True)

In [55]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


catboost = CatBoostClassifier(iterations=1000, 
                               learning_rate=0.1,
                               depth=6,
                               loss_function='Logloss',
                               verbose=False)


catboost.fit(X, y)

submission.loc[test.index, "moved_after_2019"] = catboost.predict(test)

submission["moved_after_2019"] = submission["moved_after_2019"].astype(int)
submission["moved_after_2019"].value_counts()

"""# Test setinde modelin performansını değerlendir
y_pred = catboost.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
"""

'# Test setinde modelin performansını değerlendir\ny_pred = catboost.predict(X_test)\naccuracy = accuracy_score(Y_test, y_pred)\nprint(f"Accuracy: {accuracy:.2f}")\n'

In [56]:
submission.to_csv('submission.csv')